In [ ]:
# ! pip install sqlalchemy
# ! rm /kaggle/working/sqlite3.db
!pip install scipy==1.4.1

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import scikitplot as skplt
import cv2
import copy
from PIL import Image
import pydicom
import scipy.ndimage
import imageio
from os import listdir
from sqlalchemy import create_engine
import sqlite3
from tqdm import tqdm
from torchvision import transforms
import torch
import random
from torch.utils import data
from sklearn.metrics import roc_auc_score
import torchvision.models as models
from torch.utils.data import Dataset
import pydicom as dicom
import time
import pickle
from sklearn.model_selection import train_test_split
path = "../input/rsna-str-pulmonary-embolism-detection/train/"
train = pd.read_csv("../input/rsna-str-pulmonary-embolism-detection/train.csv")
train=train[["StudyInstanceUID","SeriesInstanceUID","SOPInstanceUID","pe_present_on_image"]]
# pbar = range(0,train.shape[0])

# pbar = tqdm(pbar, dynamic_ncols=True, smoothing=0.01)
# bad_rows=[]
# for i in pbar:
# #     try:
#     ds=dicom.dcmread(path+train.iloc[i,0]+'/'+train.iloc[i,1]+'/'+train.iloc[i,2]+'.dcm')
#     dcm_sample=ds.pixel_array
# #     except Exception:
# #         print()
# #         bad_rows.append(i)
# train=train.drop(bad_rows)
        
print(train.head())
train=train[:200000]
a = train["StudyInstanceUID"]=="0038fd5f09f5"
print(len(train[a]))
print(train[a]["pe_present_on_image"].sum())
train_dataset, test_dataset = train_test_split(train, test_size=0.25, random_state=42)

class myNet(torch.nn.Module):
    def __init__(self):
#         super(myNet, self).__init__()
        super().__init__()

#         model = torch.hub.load('pytorch/vision:v0.6.0', 'inception_v3', pretrained=False)
        model=models.inception_v3(pretrained=True,aux_logits=False)
        self.model=model

#         print(self.inception_layer)
#         print(list(self.inception_layer.children()))

        self.Linear_layer = torch.nn.Linear(1000, 1)
        self.logistic_layer=torch.nn.Sigmoid()
        
    def forward(self, x):
#         x = self.inception_layer(x)
        x=self.model(x)
#         print(x.logits)
        x = self.Linear_layer(x)
#         print(x)
        x = self.logistic_layer(x)
        
        return x



preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def transform_to_hu(image):
    pixel_array=copy.deepcopy(image.pixel_array)
    pixel_array = pixel_array.astype(np.int16)
    pixel_array[pixel_array <= -1000] = 0

    intercept = image.RescaleIntercept
    slope = image.RescaleSlope
        
    if slope != 1:
        pixel_array = slope * pixel_array.astype(np.float64)
        pixel_array = pixel_array.astype(np.int16)
            
    pixel_array += np.int16(intercept)
    return pixel_array


class PEDataset(Dataset):
    def __init__(self, transform,pd):
        self.transform = transform
        self.df=pd
        
    def __len__(self):
        return self.df.shape[0]
    def _load_image_from_index(self, index):
        ds=dicom.dcmread(path+self.df.iloc[index,0]+'/'+self.df.iloc[index,1]+'/'+self.df.iloc[index,2]+'.dcm')
        pixel_array=transform_to_hu(ds)
#         print(ds.RescaleIntercept)
#         print(pixel_array)
#         print(pixel_array.max())
#         print(pixel_array.min())
#         print(transform_to_hu(ds))
#         print(transform_to_hu(ds).max())
#         print(transform_to_hu(ds).min())
#         plt.figure()

#         plt.subplot(1,2,1)
#         plt.imshow(ds.pixel_array,cmap="Greys_r")
#         plt.subplot(1,2,2)
#         plt.imshow(transform_to_hu(ds),cmap="Greys_r")
#         plt.show()
#         pixel_array=transform_to_hu(ds)
        _max=pixel_array.max()
        _min=pixel_array.min()
        temp=Image.fromarray((255 * (pixel_array+0.0 - _min) / (_max+0.0 - _min)).astype(np.uint8)).convert("RGB")
#         print(temp)
#         plt.figure()
#         plt.subplot(1,2,1)
#         plt.imshow(temp)
#         plt.subplot(1,2,2)
#         plt.imshow(transform_to_hu(ds),cmap="Greys_r")
#         plt.show()
        img=self.transform(temp)

        return img,self.df.iloc[index,3]
    def __getitem__(self, index):
        try:
            img,label = self._load_image_from_index(index)
        except:
            index=random.random()*self.__len__()
            img,label = self.__getitem__(int(index))
        

        return img,label

def data_sampler(dataset, shuffle):
    if shuffle:
        return data.RandomSampler(dataset)
    else:
        return data.SequentialSampler(dataset)    
dataloaders={}
train_dataset=PEDataset(preprocess,train_dataset)
test_dataset=PEDataset(preprocess,test_dataset)
train_loader= data.DataLoader(train_dataset,batch_size=64,sampler=data_sampler(train_dataset, shuffle=True),drop_last=True)
test_loader= data.DataLoader(test_dataset,batch_size=64,sampler=data_sampler(test_dataset, shuffle=True),drop_last=True)

dataloaders["train"]=train_loader
dataloaders["val"]=test_loader
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    print("begin trainning")
    since = time.time()

    val_acc_history = []
    loss_dur_train = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
 
    for epoch in tqdm(range(num_epochs), dynamic_ncols=True, smoothing=0.01):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        pred_result = None
        true_result = None
        for phase in ['val','train']:
            cmt =  torch.zeros(2,2, dtype=torch.int32)
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloaders[phase]):
#                 print(inputs.size())
                inputs = inputs.cuda()
                labels = labels.cuda()

                optimizer.zero_grad()


                with torch.set_grad_enabled(phase == 'train'):

                    if is_inception and phase == 'train':

                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
#                         print(outputs)
#                         print(labels)
#                         loss = criterion(outputs, labels)
#                         print(loss)
                    labels=labels.to(torch.float32)
                    
                    preds = torch.round(outputs).squeeze(1)
                    if pred_result is None:
                        pred_result = outputs.squeeze(1)
                        true_result = labels
                    else:
                        pred_result = torch.cat((pred_result,outputs.squeeze(1)))
                        true_result = torch.cat((true_result,labels))
                    loss = criterion(outputs.squeeze(1), labels)
                    loss_dur_train.append(loss.mean().item())
#                     print("qqq")
#                     print(preds.size())
#                     print(labels.size())
#                     print(torch.sum(preds == labels.data))
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.mean().backward()
                        optimizer.step()
                
                running_loss += loss.mean().item() * inputs.size(0)
                
                running_corrects += torch.sum(preds == labels.data)
#             print(running_corrects.double())
#             print(len(dataloaders[phase].dataset))
                stacked = torch.stack((preds,labels.data),dim=1)
                for p in stacked:
                    tl, pl = p.tolist()
                    cmt[int(tl),int(pl)] = cmt[int(tl),int(pl)] + 1
            if phase == 'train':
                print('--------------trainning loss---------')
                print(loss_dur_train)
            if phase=='val':
                print("---------------roc----------------")
                print(pred_result.size())
                print(true_result.size())
                print(roc_auc_score(true_result.cpu(), pred_result.cpu()))

                fpr, tpr, threshold = skplt.metrics.roc_curve(true_result.cpu(), pred_result.cpu())
                roc_auc = skplt.metrics.auc(fpr, tpr)
                plt.title('Receiver Operating Characteristic')
                plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
                plt.legend(loc = 'lower right')
                plt.plot([0, 1], [0, 1],'r--')
                plt.xlim([0, 1])
                plt.ylim([0, 1])
                plt.ylabel('True Positive Rate')
                plt.xlabel('False Positive Rate')
                plt.show()

            print(cmt)
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            torch.save(
                    {
                        "model": model.state_dict(),
                    },"checkpoint.pt",
                )
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

 
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history
mynet=myNet()
# ckpt = torch.load("/kaggle/working/checkpoint.pt")
# mynet.load_state_dict(ckpt["model"])
optimizer = torch.optim.SGD(mynet.parameters(), lr=0.01)
criterion = torch.nn.BCELoss()
model, val = train_model(mynet.cuda(), dataloaders, criterion, optimizer, num_epochs=2, is_inception=False)
with open("model.pkl",'ab') as f:
    pickle.dump(model, f)




In [ ]:
# path = "../input/rsna-str-pulmonary-embolism-detection/"
# train = pd.read_csv(path + "train.csv")
# test = pd.read_csv(path + "test.csv")
# train.to_sql('ct_train', engine)
# test.to_sql('ct_test', engine)


# conn = sqlite3.connect('sqlite3.db')
# cursor = conn.cursor()

# cursor.execute('PRAGMA table_info(ct_train)')

# values = cursor.fetchall()
# for value in values:
#     print(value)

In [ ]:
# def load_slice(path):
#     slices = [pydicom.read_file(path + '/' + s) for s in listdir(path)]
#     slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
#     try:
#         slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
#     except:
#         slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
#     for s in slices:
#         s.SliceThickness = slice_thickness
        
#     return slices


    
#     return np.array(images, dtype=np.int16)
# first_patient = load_slice('../input/rsna-str-pulmonary-embolism-detection/train/0003b3d648eb/d2b2960c2bbf')
# first_patient_pixels = transform_to_hu(first_patient)
# # print(first_patient_pixels)
# def sample_stack(stack, rows=7, cols=7, start_with=10, show_every=4):
#     fig,ax = plt.subplots(rows,cols,figsize=[18,20])
#     for i in range(rows*cols):
#         ind = start_with + i*show_every
#         ax[int(i/rows),int(i % rows)].set_title(f'slice {ind}')
#         ax[int(i/rows),int(i % rows)].imshow(stack[ind],cmap='bone')
#         ax[int(i/rows),int(i % rows)].axis('off')
#     plt.show()

# sample_stack(first_patient_pixels)


In [ ]:
# print(train.loc[train["SeriesInstanceUID"]=="d2b2960c2bbf"]["SOPInstanceUID"])
# # print(train.loc[1398587:1398809,"negative_exam_for_pe"])
# print(train.shape)
# print(train.loc[train["negative_exam_for_pe"]==0].shape)

In [ ]:
# list1 = train["SeriesInstanceUID"].values.tolist()
# print(np.unique(list1, return_counts=True))
# dct=dict(zip(*np.unique(list1, return_counts=True)))
# lst=sorted(list(dct.values()))
# temp=np.unique(lst, return_counts=True)

# fig = plt.figure() 
  
# plt.ylabel("Count") 
# plt.xlabel("Number of CT images per case") 
# plt.ylim([0,150])
# plt.bar(temp[0], temp[1], color ='maroon') 
# plt.show()